In [ ]:
from google.colab import files
uploaded = files.upload()


Saving yelp.csv to yelp.csv


In [ ]:
import pandas as pd

df = pd.read_csv("yelp.csv")
df.head()


,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [ ]:
!ls


sample_data  yelp.csv


In [ ]:
df.columns


Index(['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id',
       'cool', 'useful', 'funny'],
      dtype='object')

In [ ]:
df = df[['text', 'stars']]
df = df.sample(200, random_state=42).reset_index(drop=True)

df.head()


,text,stars
0,We got here around midnight last Friday... the...,4
1,Brought a friend from Louisiana here. She say...,5
2,"Every friday, my dad and I eat here. We order ...",3
3,"My husband and I were really, really disappoin...",1
4,Love this place! Was in phoenix 3 weeks for w...,5


In [ ]:
!pip install google-generativeai tqdm


In [ ]:
!pip install openai


In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXX"
os.environ["OPENAI_API_BASE"] = ""

client = OpenAI()


In [ ]:
import os
print(os.environ.get("OPENAI_API_KEY") is not None)


True


In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENAI_API_KEY"]
)

response = client.chat.completions.create(
    model="mistralai/mistral-7b-instruct",
    messages=[{"role": "user", "content": "Reply with exactly the word OK"}],
    temperature=0
)

print(response)


ChatCompletion(id='gen-1765717530-D19KaSHnUd3I0FR36Bk3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' ', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1765717530, model='mistralai/mistral-7b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=19, total_tokens=22, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, video_tokens=0), cost=6.94e-07, is_byok=False, cost_details={'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 5.32e-07, 'upstream_inference_completions_cost': 1.62e-07}), provider='DeepInfra')


In [ ]:
print(response.choices[0].message.content)


In [ ]:
def call_llm(prompt):
    try:
        response = client.chat.completions.create(
            model="mistralai/mistral-7b-instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        text = response.choices[0].message.content
        # Remove known junk tokens safely
        text = text.replace("<s>", "").replace("</s>", "").strip()
        return text
    except Exception as e:
        print("ERROR:", e)
        return None



In [ ]:
call_llm("Reply only OK")


'OK'

In [ ]:
import json

def parse_json(output):
    try:
        return json.loads(output), True
    except:
        return None, False



In [ ]:
def prompt_v1(review):
    return f"""
    Classify the following restaurant review into a star rating from 1 to 5.

    Review:
    {review}
    """


In [ ]:
def prompt_v2(review):
    return f"""
    Predict a star rating (1–5) for the review.

    Return ONLY valid JSON:
    {{
      "predicted_stars": 4,
      "explanation": "Short reason"
    }}

    Review:
    {review}
    """


In [ ]:
def prompt_v3(review):
    return f"""
Please classify the following Yelp review into a star rating from 1 to 5.

Respond ONLY in valid JSON with this format:
{{
  "predicted_stars": 4,
  "explanation": "Brief reasoning for the assigned rating."
}}

Review:
{review}
"""




In [ ]:
prompt_v1, prompt_v2, prompt_v3


(<function __main__.prompt_v1(review)>,
 <function __main__.prompt_v2(review)>,
 <function __main__.prompt_v3(review)>)

In [ ]:
acc1, json1 = evaluate(prompt_v1)
print(acc1, json1)


100%|██████████| 200/200 [01:49<00:00,  1.83it/s]

0 0.0


In [ ]:
acc2, json2 = evaluate(prompt_v2)
acc3, json3 = evaluate(prompt_v3)

import pandas as pd
pd.DataFrame({
    "Prompt Version": ["V1 - Simple", "V2 - JSON", "V3 - Constrained"],
    "Accuracy": [acc1, acc2, acc3],
    "JSON Validity Rate": [json1, json2, json3]
})


100%|██████████| 200/200 [00:59<00:00,  3.39it/s]


,Prompt Version,Accuracy,JSON Validity Rate
0,V1 - Simple,0.000000,0.000
1,V2 - JSON,0.333333,0.060
2,V3 - Constrained,0.538462,0.065


In [ ]:
output = call_llm(prompt_v3(df.loc[0, "text"]))
print(output)




{
  "predicted_stars": 4,
  "explanation": "The review mentions that the restaurant was empty late on a Friday, but they still served good food, had friendly service, and quality cocktails. The reviewer also notes that it's a great spot for happy hour and stays busy until 10pm. Additionally, the reviewer mentions a positive update about the patio being great for day-drinking on weekends."
}
